<h3 align="center"><b>Filesystem Alert</h3>

***

#### **Bash:**

In [3]:
%%bash

df -H | grep -Po "^/dev/(?!.*snap).*" | while read i
do
percent=$(echo $i | awk 'gsub("%", ""){print $5}')
fs=$(echo $i | awk '{print $1}')

if (( $percent >= 60 )); then
echo "Running out of space $fs $percent% on $(hostname)."
fi
done

Running out of space /dev/sda5 67% on joe.


***

#### **Python:**

#### **Using statvfs System Call**

In [4]:
%%python3

from os import statvfs as svfs
from re import compile, search
from socket import gethostname
from pathlib import Path

class DiskUsage:
    def __init__(self, file):
        self.f_blocks = svfs(file).f_blocks
        self.f_bfree = svfs(file).f_bfree

    def usep(self):
        return f"{(self.f_blocks - self.f_bfree)/self.f_blocks:.0%}"

comp = compile(r'^/dev/(?!.*snap)')
mline = Path('/proc/mounts').read_text().splitlines()
fs = [i.split()[:2] for i in mline if comp.search(i)]
     
for x in fs:
    disk = DiskUsage(x[1])
    if int(disk.usep().replace('%','')) > 60:
        print(f'Running out of space {x[0]} {disk.usep()}',end = ' ')
        print(f'on {gethostname()}.')

Running out of space /dev/sda5 66% on joe.


***

#### **Using New Subprocess API** 

In [10]:
%%python3

from subprocess import PIPE, run
from socket import gethostname
from re import compile, search


cmd = 'df -h'
comp = compile(r'^/dev/(?!.*snap).*')
args = dict(stdin=PIPE, stdout=PIPE, stderr=PIPE)
stdout = run(cmd.split(), **args).stdout.decode()
fs = [i for i in stdout.splitlines() if comp.search(i)]

for i in fs:
    percent = int(i.split()[4].replace('%',''))
    dev = i.split()[0]
    if percent > 50:
        print(f'Running out of space {dev} {percent}% on {gethostname()}.')

Running out of space /dev/sda5 67% on joe.


***

#### **Using old Subprocess API**

In [11]:
%%python3

from subprocess import check_output
from socket import gethostname
from re import search

cmd = 'df -h'
out = check_output(cmd.split()).decode().splitlines()
fs = [i for i in out if search(r'^/dev/(?!.*snap)',i)]

for i in fs:
    percent = int(i.split()[4].replace('%',''))
    dev = i.split()[0]
    if percent > 50:
        print(f'Running out of space {dev} {percent}% on {gethostname()}.')

Running out of space /dev/sda5 67% on joe.
